In [75]:
import gauleg as gl 
import sympy as sp 
import numpy as np 
import pandas as pd 
import math
import matplotlib.pyplot as plt
import Solver as sl 
from scipy.interpolate import interp1d

%load_ext autoreload

%autoreload 2
rng = np.random.default_rng(seed=42)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Generating Delta Vector

In [76]:
def interpolation_for_u_h(nodal, mesh):
    """
    Takes in the solution of the forward solver at high resolution and returns the 
    corresponing values of u_h at a set xi that is predetermined 

    Returns: 
    Array of observation u_h that corresponds to each xi using interpolation if xi not 
    in final mesh. 
    """
    nodal = sl.assemble_nodal_values(nodal)
    mesh = np.array(mesh)  # Replace with your mesh node coordinates
    nodal = np.array(nodal).flatten()  # Replace with your computed u_h values   
    x_min = 0.0
    x_max = 1.0
    num_points = 100  # Number of xi's
    xi = np.linspace(x_min, x_max, num_points)

    u_h_interp = interp1d(mesh, nodal, kind='linear', fill_value="extrapolate")
    
    # Evaluate u_h at the observation points
    u_h_at_xi = u_h_interp(xi)

    return u_h_at_xi



In [77]:
def add_noise(observations_at_xi, num_points):
    """
    Adds a normally distributed noise, theta
    to observations from the forward solver.

    Arguments:
    observations_at_xi : observations at predetermined xi using interplotion. 
    num_points : how big your covariance matrix is 

    Returns:
    Delta : Array of Noisy observations.
    
    """
    
    cov_matrix = np.eye(num_points)
    noise = np.random.multivariate_normal(np.zeros(num_points), cov_matrix)
    delta = observations_at_xi + noise 
    return delta

beta_true = np.array([0.5,1/6])
mesh_true, c_sol_true = sl.refinement_loop(0.00001, beta_true)
g_beta_true = interpolation_for_u_h(c_sol_true, mesh_true)
delta = add_noise(g_beta_true, num_points=100)
delta

array([-0.55919345, -0.6145587 , -1.09010412,  2.38249048,  3.02493759,
        0.43175515,  0.2852813 ,  0.28758583, -0.17899745,  2.03503093,
       -0.03985959, -0.23638335,  1.65275107, -0.36141127,  1.54502608,
        0.28046822,  0.88794092,  3.00923659,  3.6196285 , -0.64206682,
        1.86491816,  0.77940068,  1.26302852,  1.90922779, -1.41353119,
        1.76213596,  0.4079655 , -0.137167  ,  1.67737621,  0.67490178,
        1.45302718,  1.47837982,  0.23307743,  1.54074054,  2.03165078,
        0.24264943,  2.23000808,  1.9667369 ,  1.06896801,  2.31561316,
        0.80001985,  1.74821331,  0.71945228,  2.69360711,  3.34186017,
        2.39524474,  1.00445196,  2.5381245 ,  2.21564714,  1.75248151,
        4.23989019,  1.82236055,  1.26986904,  0.5798827 ,  2.03354983,
        1.57865493,  1.300643  ,  0.98951445,  0.03903511,  1.30055141,
        1.74024847,  1.22092033,  1.92470113,  1.92681925,  1.36676167,
        1.22681873,  1.69655131,  1.70847244,  1.55829537,  0.10

# Define Likelihood Function 

In [78]:
def phi(obseravations, predicted, sigma = 0.01) :
    '''
    For a set of predetermined points xi -- obtained via np.linspace,
    this function defines the likelihood function 

    Arguments:
    observations: Generated noisy observation using beta_true -- corresponds to y in literature
    predicted: For a proposed beta_i, we compute the noisy observation using the forward solver 
    -- corresponds to g(beta_i) in literature

    Returns: 
    Likelihood function that is proportional to the prior distribution
    
    '''
    misfit = np.sum((obseravations - predicted) ** 2)
    return np.exp(-np.sum(misfit**2) / (2 * sigma**2))





# MCMC algorithm 

In [ ]:
def MCMC(observations, number_of_iter, burn_in): 
    '''
    Builds a Markov Chain 

    Key Steps:
    1. Initialise a choice of Beta, beta_0 
    2. Compute likelihood of beta_0, using delta and beta_0_predicted
    3. Initialise the loop.
        - we propose a new beta_i from x* ~ Uniform(0.15, 0.85) and r ~ Uniform(0, 0.15)
        - compute y_i and g(beta_i)
        - compute likelihood using {y_i and g(beta_i)}
        - set alpha = min{1, likelihood }
    '''
    # range of uniform distribution 
    x_star_range = (0.15, 0.85)
    r_range = (0, 0.15)

    # intiailise markov chain 
    chain = [] 
    beta_init = np.array([0.5, 1/4])
    beta_current = beta_init.copy()
    iter_count = 0
    acceptance_count = 0 
    acceptance_prob_history = []

    # initialise current observations and likelihood 
    mesh_current, c_sol_current = sl.refinement_loop(0.00001, beta = beta_current)
    y_current = interpolation_for_u_h(c_sol_current, mesh_current)
    likelihood_current = phi(observations, y_current)
    
    for i in range(number_of_iter):
        beta_proposal = np.array([
            np.random.uniform(*x_star_range),
            np.random.uniform(*r_range)
        ])
        mesh_proposal, c_sol_proposal = sl.refinement_loop(0.00001, beta = beta_proposal)
        y_proposal = interpolation_for_u_h(c_sol_proposal, mesh_proposal)
        likelihood_proposal = phi(observations, y_proposal)

        # compute acceptance probability 
        acceptance_prob = min(1, likelihood_proposal/likelihood_current)
        acceptance_prob_history.append(acceptance_prob)
        
        # Accept or reject the proposal
        if np.random.rand() < acceptance_prob:
            beta_current = beta_proposal
            likelihood_current = likelihood_proposal
            acceptance_count += 1
        
        # Append the current beta to the chain
        chain.append(beta_current.copy())
        iter_count += 1
        print(iter_count)
    
    chain = np.array(chain)
    # Discard burn-in and compute the MCMC estimate as the mean of the samples
    beta_mcmc = np.mean(chain[burn_in:], axis=0)
    return chain, beta_mcmc, acceptance_prob_history, acceptance_count

In [80]:
chain, beta_mcmc, acceptance_probablity_list, acceptance_count = MCMC(observations = delta, number_of_iter = 5000, burn_in=1000)
print("True beta:", beta_true )
print("MCMC estimated beta:", beta_mcmc)
print("Acceptance Probability History:", acceptance_probablity_list)
print("Acceptance Count:", acceptance_count)

C:\Users\Admin\AppData\Local\Temp\ipykernel_2196\2261265667.py:41: RuntimeWarning: invalid value encountered in scalar divide
  acceptance_prob = min(1, likelihood_proposal/likelihood_current)


1
2


KeyboardInterrupt: 